### **Classes** 

We have already seen classes in c++. The general idea remains the same - bundle together certain data and functions that operate on it. We just need to learn the new syntax.

In [7]:
class MyFirstClass: #syntax to define a class
    """ My First Class documentation """
    some_variable = 757 #member variable
    
    def greet(self): #member function
         print("Hello, World, I'm a useless class!")

In [9]:
myvar = MyFirstClass() #create a class instance
print(myvar.some_variable) #print the data
myvar.greet() #call a member fucntion
myvar.some_variable = 234 #change a member variable
print(myvar.some_variable)
myvar.some_variable = "can change to string" #change a member variable to a different type
print(myvar.some_variable)

757
Hello, World, I'm a useless class!
234
can change to string


**big syntax surprise for c++ programmers:**

In [10]:
myvar2=MyFirstClass()
print(myvar.some_variable)

can change to string


As we can see, the value became shared between instances. This is called a static variable. it can be used, for example, to count the class objects or to keep some commom constants. However, let's look at how to create "normal" variables:

In [11]:
 class MySecondClass:
    """ My First Class documentation """    
    def __init__(self, number):
        self.number = number

Note that `self` is obligatory as a first argument to all class functions. On the bright side, you can reuse names of member variables for initialization parameters. Also note that the `init` methode is not really a constructor the way we are used to think of them in c++ as the object is already constructed when it's called.

In [12]:
myvar3=MySecondClass(5)
myvar4=MySecondClass(9)
print(myvar3.number)
print(myvar4.number)

5
9


In [13]:
myvar3.number=99
myvar4.number=111
print(myvar3.number)
print(myvar4.number)

99
111


**more surpises:**

In [21]:
myvar3.new_var=8 #this was not defined in a class, don't do that
myvar4.new_var=7
print(myvar3.new_var)
print(myvar4.new_var)

8
7


In [17]:
myvar.new_var=0.1
myvar2.new_var=0.2
print(myvar.new_var)
print(myvar2.new_var)

0.1
0.2


In [19]:
myvar5=myvar4 #just an extra label, not a copy
print(myvar5.number)
print(myvar5.new_var)

111
7


In [22]:
myvar4.number="hi" 
print(myvar5.number)

hi


In [23]:
print(myvar4) #no actual info, need to define a function for this

In [33]:
class MyThirdClass:
    def __init__(self,a,b):
        self.a=a
        self.b=b
        
    def print(self): #non pythonic way, avoid it
        print(a,b)
        
    def __str__(self):
        return "{0},{1}".format(self.a, self.b) #"converts" class to a string

In [34]:
print(MyThirdClass(22,33))

22,33


In [37]:
help(MyThirdClass)

Help on class MyThirdClass in module __main__:

class MyThirdClass(builtins.object)
 |  MyThirdClass(a, b)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, a, b)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  print(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [40]:
var1=MyThirdClass(11,22)
var2=MyThirdClass(22,33)
print(var1+var2) #error

TypeError: unsupported operand type(s) for +: 'MyThirdClass' and 'MyThirdClass'

**Redefining operators:**

In [46]:
class MyThirdClass:
    def __init__(self,a,b):
        self.a=a
        self.b=b
        
    def print(self): #non pythonic way, avoid it
        print(a,b)
        
    def __add__(self,other):   
        a=self.a+other.a
        b=self.b+other.b
        return MyThirdClass(a,b)
        
    def __str__(self):
        return "{0},{1}".format(self.a, self.b) #"converts" class to a string

In [47]:
#note that you have to redefine the variables, or they are not updated after we updated the class
var1=MyThirdClass(11,22)
var2=MyThirdClass(22,33)
print(var1+var2) 

33,55


## **Mixing the languages: Cython**

One of the ways to speed up python code is to convert it into compiled C code. There are a number of ways to do it, let's start with `cython`, do `conda install cython` inside your environment to install it.



In [48]:
%load_ext Cython

In [49]:
def fib1(N):
    a,b = 0,1
    for i in range(N):
        a,b = b,a+b
    return a


In [51]:
%%timeit 
fib1(10000)

1.84 ms ± 41.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
%%cython
def fib2(N):
    a,b = 0,1
    for i in range(N):
        a,b = b,a+b
    return a

In [53]:
%%timeit 
fib2(10000)

1.6 ms ± 7.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


We didn't get much speed up, but still some, just for using `cythion` magic in the cell. Now let's remember that variables can have types:

In [57]:
%%cython
def fib3(int N):
    cdef int i
    cdef int a=0,b=1
    for i in range(N):
        a,b = b,a+b
    return a


In [55]:
%%timeit
fib3(10000)

7.06 µs ± 46.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Impressive, isn't it? Roughly a 230 times speed up just for using variables with types! 
We can actually see the generated `c` code if we add `-a` to the magic command:

In [58]:
%%cython -a
def fib3(int N):
    cdef int i
    cdef int a=0,b=1
    for i in range(N):
        a,b = b,a+b
    return a


**Cython types**:

In [59]:
%%cython
cdef char i=1           # Oddly an 8 bit integer (-128 to 127) 
cdef short j=2          # 16 bit integer (-32,768 to 32,767)
cdef int k=3            # 32 bit integer (-2,147,483,648 to 2,147,483,647)
cdef unsigned int l=4   # 32 bit +ve integer (0 to 4,294,967,295), "unsigned" can go infront of all numeric types
cdef long int m=5       # 64 bit integer (-9,223,372,036,854,775,808 to 9,223,372,036,854,775,807)
cdef float x=0.0        # 32 bit float (6 decimal places, max exponent 38)
cdef double y = 0e0     # 64 bit float (12 decimal places, max exponent 1023)
cdef list list1 = [1,2,3]       # just a normal list (not much performance gain)

Of course, this means we need to worry about overflow again:

In [60]:
%%cython
cdef short j
j = 200**2
print(j)

-25536


In [61]:
%%cython
cdef unsigned int j
j = -1
print(j)


4294967295


Strings are stored completely differently in C, so cdef doesn't help. It's best just to keep strings as python variables.

In [63]:
%%cython
#an example of if you need C strings
def test(input):
    input_byte = input.encode('utf-8')
    cdef char* c_string = input_byte
    cdef bytes py_string_byte = c_string
    output = py_string_byte.decode('utf-8')
    print(output)
test('Hello')

Hello


We can also use standard c math library:

In [70]:
%%cython
from libc.math cimport sin

def sin_c(double x):
    return sin(x)

In [71]:
import math
x = 0.5
%timeit math.sin(x)
%timeit sin_c(x)

109 ns ± 5.92 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
63 ns ± 2.4 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


You can mix cython and numpy arrays, but numpy is already in c, so the speed up won't be too big.

**Cython for scripts and some introduction to creating modules:**

Put your cython code in a file with extension `.pyx` like `cython_module.pyx`: (this will actually create a file for us)

In [73]:
%%file cython_module.pyx
"""
Cython code for fibonnaci numbers
"""
cpdef int fibonacci(int N):
    cdef int i
    cdef int a=0,b=1
    for i in range(N):
        a,b = b,a+b
    return a


Writing cython_module.pyx


Create a file called setup.py with the following:

In [81]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("cython_module.pyx",language_level = "3")
)


Overwriting setup.py


compile the code:

In [82]:
%%bash
python3 setup.py build_ext --inplace

running build_ext


Now you can use the new functions:

In [83]:
import cython_module as cym
cym.fibonacci(10)

55

**We will come back to cython in the next lecture, this was just an introduction to give you the "taste"**
The rest of this lecture is devoted to `pibind` and can be found in the folder `pybind11`